# YOLO bounding boxes: Example 1

In this notebook, we train a YOLO model to reconize objects defined as [ROIs](https://github.com/sgoldenlab/simba/blob/master/docs/ROI_tutorial_new.md) in SimBA.

Specifically, we train an ROI model to recognize two different objects in a rat novel-object recognition task. 



In [12]:
!pip show ultralytics

Name: ultralytics
Version: 8.3.19
Summary: Ultralytics YOLO 🚀 for SOTA object detection, multi-object tracking, instance segmentation, pose estimation and image classification.
Home-page: https://ultralytics.com
Author: 
Author-email: Glenn Jocher <glenn.jocher@ultralytics.com>, Jing Qiu <jing.qiu@ultralytics.com>
License: AGPL-3.0
Location: /home/simon/miniconda3/envs/simba_310/lib/python3.10/site-packages
Requires: matplotlib, numpy, opencv-python, pandas, pillow, psutil, py-cpuinfo, pyyaml, requests, scipy, seaborn, torch, torchvision, tqdm, ultralytics-thop
Required-by: 


In [2]:
!pip show simba-uw-tf-dev

Name: Simba-UW-tf-dev
Version: 2.3.4
Summary: Toolkit for computer classification and analysis of behaviors in experimental animals
Home-page: https://github.com/sgoldenlab/simba
Author: Simon Nilsson, Jia Jie Choong, Sophia Hwang
Author-email: sronilsson@gmail.com
License: GNU General Public License v3 (GPLv3)
Location: /home/simon/miniconda3/envs/simba_310/lib/python3.10/site-packages
Requires: cefpython3, dash, dash-color-picker, dash-colorscales, dash-core-components, dash-html-components, dtreeviz, eli5, ffmpeg-python, geos, graphviz, h5py, imbalanced-learn, imgaug, imutils, joblib, kaleido, matplotlib, numba, numexpr, numpy, opencv-python, pandas, Pillow, plotly, psutil, pyarrow, pyglet, pyyaml, scikit-image, scikit-learn, scipy, seaborn, shap, shapely, statsmodels, tables, tabulate, tqdm, trafaret, xgboost, xlrd, yellowbrick
Required-by: 


In [9]:
import os
from simba.bounding_box_tools.yolo.model import fit_yolo, inference_yolo
from simba.bounding_box_tools.yolo.visualize import YOLOVisualizer
from simba.third_party_label_appenders.converters import simba_rois_to_yolo, split_yolo_train_test_val
from ipywidgets import Video
from IPython.display import HTML

In [4]:
# FIRST WE CREATE SOME YOLO FORMATED DATA LABELS AND EXTRACT THEIR ASSOCIATED IMAGES FROM THE VIDEOS AND SIMBA ROI DEFINITIONS
ROI_PATH = r"/mnt/c/troubleshooting/RAT_NOR/project_folder/logs/measures/ROI_definitions.h5"
VIDEO_DIR = r'/mnt/c/troubleshooting/RAT_NOR/project_folder/videos' #DIRECTORY HOLDING THE VIDEOS WITH DEFINED ROI'S
YOLO_SAVE_DIR = r"/mnt/c/troubleshooting/yolo_data" # DIRECTORY WHICH SHOULD STORE THE YOLO FORMATED DATA
VIDEO_FRAME_COUNT = 200 # THE NUMBER OF LABELS WE SHOULD CREATE FROM EACH VIDEO
GREYSCALE = False #IF THE VIDEOS ARE IN COLOR, CONVERT  THE YOLO IMAGES TO GREYSCALE OR NOT

#WE CREATE THE YOLO DATASET BASED ON THE ABOVE DEFINITIONS
simba_rois_to_yolo(roi_path=ROI_PATH, video_dir=VIDEO_DIR, save_dir=YOLO_SAVE_DIR, roi_frm_cnt=VIDEO_FRAME_COUNT, greyscale=GREYSCALE, verbose=True, obb=True)

Reading in ROI geometries...
Reading ROI geometries for video 03152021_NOB_IOT_8... (1/3)
Reading ROI geometries for video 08092021_DOT_Rat9_10(2)... (2/3)
Reading ROI geometries for video 08102021_DOT_Rat7_8(2)... (3/3)
Reading ROI coordinates ...
Reading ROI coordinates for video 03152021_NOB_IOT_8... (1/3)
Reading ROI coordinates for video 08092021_DOT_Rat9_10(2)... (2/3)
Reading ROI coordinates for video 08102021_DOT_Rat7_8(2)... (3/3)
Reading ROI images ...
Reading ROI images for video 03152021_NOB_IOT_8... (1/3)
Reading ROI images for video 08092021_DOT_Rat9_10(2)... (2/3)
Reading ROI images for video 08102021_DOT_Rat7_8(2)... (3/3)
SIMBA COMPLETE: yolo ROI data saved in /mnt/c/troubleshooting/yolo_data (elapsed time: 28.9579s) 	complete


In [5]:
#NEXT, WE SPLIT THE DATA CREATED IN THE PRIOR CELL INTO DATA FOR TRAINING, TESTING AND VALIDATION.
YOLO_TRAINING_DIR = r"/mnt/c/troubleshooting/yolo_data_split" # DIRECTORY WHERE WE SHOULD STORE THE SLIP DATA.

#WE SPLIT THE DATA BASED ON ABOVE DEFINITIONS
split_yolo_train_test_val(data_dir=YOLO_SAVE_DIR, save_dir=YOLO_TRAINING_DIR, verbose=False)

In [6]:
#NEXT, WE TRAIN A YOLO MODEL BASED ON THE SPLIT DATA
INITIAL_WEIGHTS_PATH = r'/mnt/c/troubleshooting/coco_data/weights/yolov8n-obb.pt' #SOME INITIAL WEIGHTS TO START WITH, THEY CAN BE DOWNLOADED AT https://huggingface.co/Ultralytics
MODEL_SAVE_DIR = "/mnt/c/troubleshooting/yolo_mdl" #DIRECTORY WHERE TO SAVE THE TRAINED MODEL AND PERFORMANCE STATISTICS
EPOCHS = 25 # HOW MANY EPOCHS TO TRAIN THE MODEL FOR
BATCH = 16 # THE NUMBER OF OBSERVATIONS IN EACH ITERATION

#WE TRAIN THE MODEL BASED ON ABOVE DEFINITIONS
fit_yolo(initial_weights=INITIAL_WEIGHTS_PATH, model_yaml=os.path.join(YOLO_TRAINING_DIR, 'map.yaml'), save_path=MODEL_SAVE_DIR, epochs=EPOCHS, batch=BATCH, plots=True)

New https://pypi.org/project/ultralytics/8.3.31 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.19 🚀 Python-3.10.15 torch-2.5.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine/trainer: task=obb, mode=train, model=/mnt/c/troubleshooting/coco_data/weights/yolov8n-obb.pt, data=/mnt/c/troubleshooting/yolo_data_split/map.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=1024, save=True, save_period=-1, cache=False, device=None, workers=8, project=/mnt/c/troubleshooting/yolo_mdl, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False,

train: Scanning /mnt/c/troubleshooting/yolo_data_split/labels/train... 420 images, 0 backgrounds, 0 corrupt: 100%|██████████| 420/420 [00:01<00:00, 266.70it/s]


train: New cache created: /mnt/c/troubleshooting/yolo_data_split/labels/train.cache


val: Scanning /mnt/c/troubleshooting/yolo_data_split/labels/val... 60 images, 0 backgrounds, 0 corrupt: 100%|██████████| 60/60 [00:00<00:00, 189.84it/s]


val: New cache created: /mnt/c/troubleshooting/yolo_data_split/labels/val.cache
Plotting labels to /mnt/c/troubleshooting/yolo_mdl/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 63 weight(decay=0.0), 73 weight(decay=0.0005), 72 bias(decay=0.0)
Image sizes 1024 train, 1024 val
Using 8 dataloader workers
Logging results to /mnt/c/troubleshooting/yolo_mdl/train
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      25/25      5.23G    0.09872     0.1321     0.9384          8       1024: 100%|██████████| 27/27 [00:04<00:00,  6.40it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00, 10.21it/s]

                   all         60        120      0.999          1      0.995      0.995



25 epochs completed in 0.040 hours.
Optimizer stripped from /mnt/c/troubleshooting/yolo_mdl/train/weights/last.pt, 7.0MB
Optimizer stripped from /mnt/c/troubleshooting/yolo_mdl/train/weights/best.pt, 7.0MB

Validating /mnt/c/troubleshooting/yolo_mdl/train/weights/best.pt...
Ultralytics 8.3.19 🚀 Python-3.10.15 torch-2.5.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
YOLOv8n-obb summary (fused): 187 layers, 3,077,609 parameters, 0 gradients, 8.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.72it/s]


                   all         60        120      0.999          1      0.995      0.995
         RED_RECTANGLE         60         60      0.999          1      0.995      0.995
      YELLOW_RECTANGLE         60         60      0.999          1      0.995      0.995
Speed: 0.4ms preprocess, 4.5ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to /mnt/c/troubleshooting/yolo_mdl/train


In [7]:
#NEXT, WE USE THE TRAINED MODEL TO FIND THE ROIS IN A NEW VIDEO
INFERENCE_RESULTS = "/mnt/c/troubleshooting/yolo_results" #DIRECTORY WHERE TO STORE THE RESULTS IN CSV FORMAT
VIDEO_PATH = r'/mnt/c/troubleshooting/RAT_NOR/project_folder/videos/clipped/03152021_NOB_IOT_8_clipped.mp4' #PATH TO VIDEO TO ANALYZE
BATCH = 16 # THE NUMBER OF OBSERVATIONS (IMAGES) TO ANALYZE IN EACH ITERATION
GPU = False #IF True, THEN ANALYZE THE VIDEO FRAME ON THE GPU

inference_yolo(weights_path=os.path.join(MODEL_SAVE_DIR, 'train', 'weights', 'best.pt'), video_path=VIDEO_PATH, verbose=True, save_dir=INFERENCE_RESULTS, gpu=GPU, batch_size=BATCH)


video 1/1 (frame 1/600) /mnt/c/troubleshooting/RAT_NOR/project_folder/videos/clipped/03152021_NOB_IOT_8_clipped.mp4: 576x1024 3.6ms
video 1/1 (frame 2/600) /mnt/c/troubleshooting/RAT_NOR/project_folder/videos/clipped/03152021_NOB_IOT_8_clipped.mp4: 576x1024 3.6ms
video 1/1 (frame 3/600) /mnt/c/troubleshooting/RAT_NOR/project_folder/videos/clipped/03152021_NOB_IOT_8_clipped.mp4: 576x1024 3.6ms
video 1/1 (frame 4/600) /mnt/c/troubleshooting/RAT_NOR/project_folder/videos/clipped/03152021_NOB_IOT_8_clipped.mp4: 576x1024 3.6ms
video 1/1 (frame 5/600) /mnt/c/troubleshooting/RAT_NOR/project_folder/videos/clipped/03152021_NOB_IOT_8_clipped.mp4: 576x1024 3.6ms
video 1/1 (frame 6/600) /mnt/c/troubleshooting/RAT_NOR/project_folder/videos/clipped/03152021_NOB_IOT_8_clipped.mp4: 576x1024 3.6ms
video 1/1 (frame 7/600) /mnt/c/troubleshooting/RAT_NOR/project_folder/videos/clipped/03152021_NOB_IOT_8_clipped.mp4: 576x1024 3.6ms
video 1/1 (frame 8/600) /mnt/c/troubleshooting/RAT_NOR/project_folder/video

In [11]:
#EXPECTED RESULTS

video_url = 'https://raw.githubusercontent.com/sgoldenlab/simba/master/docs/_static/img/YOLOVisualizer.webm'
HTML(f''' <video width="600" height="600" controls> <source src="{video_url}" type="video/mp4"> </video>
''')